In [ ]:
# Generic imports
import clr
import sys
import os
from System.IO import *
from System import String
from System.Collections.Generic import List

# Needed dll for interaction with LF
sys.path.append(os.environ['LIGHTFIELD_ROOT'])
sys.path.append(os.environ['LIGHTFIELD_ROOT']+'\\AddInViews')
clr.AddReference('PrincetonInstruments.LightFieldViewV5')
clr.AddReference('PrincetonInstruments.LightField.AutomationV5')
clr.AddReference('PrincetonInstruments.LightFieldAddInSupportServices')

# PI imports
import PrincetonInstruments.LightField.AddIns as AddIns
from PrincetonInstruments.LightField.Automation import Automation
from PrincetonInstruments.LightField.AddIns import CameraSettings
from PrincetonInstruments.LightField.AddIns import DeviceType
from PrincetonInstruments.LightField.AddIns import ExperimentSettings

def set_value(setting, value):
    # Check for existence before setting
    # gain, adc rate, or adc quality
    if experiment.Exists(setting):
        experiment.SetValue(setting, value)

In [ ]:
# SPE file plotting related
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
sys.path.append("..")
from LowLevelModules.Spectroscopy import Spectrum
from LowLevelModules.GeneralFunctions import get_work_dir, prettify_plot, prettify_2d_plot

Keithley stuff

In [15]:
from pymeasure.instruments.keithley import Keithley2400
keithley = Keithley2400("GPIB0::24::INSTR")   # 'ASRL9::INSTR' if RS232 connection
keithley.apply_voltage()                      # Sets up to source voltage
keithley.source_voltage_range = 20            # Sets the source voltage range to 1 V
keithley.compliance_current = 5e-5            # Sets the compliance current to 1 mA
keithley.source_voltage = 0                   # Sets the source current to 0 mA
keithley.measure_voltage()                    # Sets up to measure voltage

Start the LightField experiment instance and check if it is OK

In [ ]:
# Create the LightField Application (true for visible)
# The 2nd parameter forces LF to load with no experiment
# find out how to load specific experiment
auto = Automation(True, List[String]())
experiment = auto.LightFieldApplication.Experiment
for device in experiment.ExperimentDevices:
        if device.Type == DeviceType.Camera:
            print("Device setup OK")

In [ ]:
import ipywidgets as widgets
from IPython.display import display

etaloning_choice = widgets.Checkbox(value=False, description='Remove CCD etaloning')
x_axis_choice = widgets.RadioButtons(options=['Wavelength (nm)', 'Energy (eV)'],
                                     value='Wavelength (nm)',
                                     description='X axis unit:')
display(etaloning_choice)
display(x_axis_choice)

In [ ]:
wdir = r'Z:\Projects\WSe2\2019\03 march\LF automation'
V_range = np.arange(-5,5)
# Enable the voltage source output
keithley.enable_source()
#Set exposure time
set_value(CameraSettings.ShutterTimingExposureTime, 200.0) #ms

#initialize the data
xdata = np.arange(1340)
ydata = V_range
zdata = np.zeros((1340, len(ydata)))

lp = VoltageScanPlot(xdata, ydata, zdata, 18, 6, 'o')
prettify_2d_plot(lp.ax, lp.cb, x_axis_choice.value, 'Voltage (V)', 'Count rate (Hz)')

for i, v in enumerate(V_range):
    # change the voltage
    keithley.ramp_to_voltage(v)
    # update file name to save data
    set_value(ExperimentSettings.FileNameGenerationBaseFileName, base_name + str(v) + "V")
    # Acquire image
    experiment.Acquire()
    # Retrieve the saved data
    filename = wdir + base_name + str(v) + "V.spe"
    data = Spectrum(filename)
    data.remove_cosmic_ray(2)
    if etaloning_choice.value:
        data.remove_etaloning(0.30, 0.46, 0.02, plotplot=False)
    if x_axis_choice.value == 'Energy (eV)':
        data.x = 1241/data.x
        data.x = data.x[::-1]
        data.y = data.y[::-1]
    if normalize_choice.value:
        data.y = data.y / np.max(data.y) + i*0.5
    
    zdata[i,:] = data.y
    lp.plot_live(zdata)  
    
keithley.disable_source()                # Disables the source output

In [ ]:
class VoltageScanPlot:
    def __init__(self, x_data, y_data, z_data, x_ext=18, y_ext=6, mk='o'):
        self.fig = plt.figure(figsize=(x_ext, y_ext))
        self.ax = self.fig.add_subplot(111)
        #self.pc = self.ax.pcolor(x_data, y_data, z_data, cmap='jet', vmin=0, vmax=np.max(z_data))
        #self.cb = self.fig.colorbar(self.pc)
        extent = [np.min(x_data), np.max(x_data), np.min(y_data), np.max(y_data)]
        self.cp = self.ax.imshow(z_data, cmap='jet', origin='center', extent=extent, 
                       interpolation='nearest', vmin=0, vmax=np.max(z_data))
        self.cb = self.fig.colorbar(self.cp, fraction=0.046, pad=0.04)
        self.ax.set_ylim([y_data[0], y_data[-1]])
        self.fig.show()
        self.fig.canvas.draw()
        self.fig.tight_layout()

    def plot_live(self, z_data):
                
        self.cb.set_data(z_data) 
        #plt.draw()
        self.fig.canvas.draw()
        #plt.pause(1e-6)
        
        